Only used for faster-RCNN

In [26]:
import helper_fxns as hf
import transforms as tr
import importlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from pylab import get_cmap
import requests
import sys
from skimage.draw import polygon
import cv2
import copy

In [24]:
importlib.reload(tr)

<module 'transforms' from 'C:\\Users\\Clinton\\Documents\\liver-mr-processor\\transforms.py'>

In [ ]:
def make_isometric(img, mrn, aug_rois, vox_dims):
    img = tr.scaley(tr.scalex(img, scale), scale)
    np.save("train_imgs\\"+mrn, img)
    
    aug_rois += [[mrn+".npy"] + [str(round(int(roi[x])*scale)) for x in range(1,5)] + roi[5:]
                for roi in rois if roi[0] == mrn+".npy"]
    
    return img, aug_rois

In [27]:
def downsample(img, mrn, aug_rois, scale=1.0):
    img = tr.scaley(tr.scalex(img, scale), scale)
    np.save("train_imgs\\"+mrn, img)
    
    aug_rois += [[mrn+".npy"] + [str(round(int(roi[x])*scale)) for x in range(1,5)] + roi[5:]
                for roi in rois if roi[0] == mrn+".npy"]
    
    return img, aug_rois

In [28]:
def aug_noise(img, mrn, aug_rois):
    mrn_rois = [x for x in rois if x[0] == mrn+".npy"]
    
    k_size = 5
    np.save("train_imgs\\"+mrn+"_b"+str(k_size), cv2.blur(img,(k_size,k_size)))
    for roi in mrn_rois:
        aug_rois.append([mrn+"_b"+str(k_size)+".npy"] + roi[1:])
    
    return aug_rois

In [29]:
def aug_scaling(img, mrn, aug_rois):
    mrn_rois = [x for x in rois if x[0] == mrn+".npy"]
    
    for scale in [1.2,1.4]:
        np.save("train_imgs\\"+mrn+"_sx"+str(scale)[-1], tr.scalex(img, scale))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_sx"+str(scale)[-1]+".npy",
                             str(round(int(roi[1])*scale)), str(round(int(roi[2])*scale))] + roi[3:])
            
        np.save("train_imgs\\"+mrn+"_sy"+str(scale)[-1], tr.scaley(img, scale))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_sy"+str(scale)[-1]+".npy"] + roi[1:3] +
                            [str(round(int(roi[3])*scale)), str(round(int(roi[4])*scale))] + roi[5:])
            
        np.save("train_imgs\\"+mrn+"_sz"+str(scale)[-1], tr.scalez(img, scale))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_sz"+str(scale)[-1]+".npy"] + roi[1:5] +
                            [str(round(int(roi[5])*scale)), str(round(int(roi[6])*scale))] + roi[7:])
    
    return aug_rois

In [30]:
def aug_rotation(img, mrn, aug_rois):
    mrn_rois = [x for x in rois if x[0] == mrn+".npy"]
    
    for angle in [15,30,345,330]:
        np.save("train_imgs\\"+mrn+"_r"+str(angle), tr.rotate(img, angle))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_r"+str(angle)+".npy"] + roi[1:])
    
    return aug_rois

In [31]:
with open("train_list.txt", "r") as f:
    rois = [x.split(',') for x in f.read().split("\n")]
aug_rois = []#[roi.split(",") for roi in rois]

In [32]:
for mrn in os.listdir("imgs\\"):
    img = np.load("imgs\\"+mrn)
    img, aug_rois = downsample(img, mrn[:-4], aug_rois)
    np.save("train_imgs\\"+mrn, img)
    aug_rois = aug_rotation(img, mrn[:-4], aug_rois)
    aug_rois = aug_scaling(img, mrn[:-4], aug_rois)
    #aug_rois = aug_noise(img, mrn[:-4], aug_rois)

In [33]:
with open('aug_train_list.txt', 'w') as f:
    for roi in aug_rois:
        f.write(",".join(roi) + "\n")